# Preprocess original census data 2017 to generate LUT for states
- Open original census data
- Extract all rows for maize
- Create LUT states
- Save file as csv

In [1]:
# Imports
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
# Paths
original_path = Path.cwd().parent / 'original_data' 
original_path

PosixPath('/home/vant/Documents/valencia/agml_workshop/inegi_censos/original_data')

In [3]:
# Replace 'file_path.xlsx' with the path to your Excel file
file_path = original_path/'ena17_ent_agri03.xlsx'

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(file_path,skiprows=5)

In [4]:
df.head()

,Entidad,Cultivo,Entidad federativa y cultivo,Superficie cultivada,Unnamed: 4,Producción,Disponibilidad del agua,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,Temporal,NaN,NaN,Riego,NaN,NaN
1,NaN,NaN,NaN,Superficie sembrada,Superficie cosechada,NaN,Superficie sembrada,Superficie cosechada,Producción,Superficie sembrada,Superficie cosechada,Producción
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Hectáreas,NaN,Toneladas,Hectáreas,NaN,Toneladas,Hectáreas,NaN,Toneladas
4,NaN,NaN,NaN,A = D + G,B = E + H,C = F + I,D,E,F,G,H,I


In [5]:
df.columns

Index(['Entidad', 'Cultivo', 'Entidad federativa y cultivo',
       'Superficie cultivada', 'Unnamed: 4', 'Producción',
       'Disponibilidad del agua', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [6]:
# Define new column names
column_names = ['Entidad federativa', 'Cultivo','Entidad federativa y cultivo','Total superficie sembrada','Total superficie cosechada',
                'Producción total','MH-temporal superficie sembrada','MH-temporal superficie cosechada','MH-temporal producción',
                'MH-riego superficie sembrada','MH-riego superficie cosechada','MH-riego producción']

# Rename the columns
df.columns = column_names

# Delete rows with initial no data
df = df.drop(index=range(5)).reset_index(drop=True)

In [7]:
df.head()

,Entidad federativa,Cultivo,Entidad federativa y cultivo,Total superficie sembrada,Total superficie cosechada,Producción total,MH-temporal superficie sembrada,MH-temporal superficie cosechada,MH-temporal producción,MH-riego superficie sembrada,MH-riego superficie cosechada,MH-riego producción
0,01 Ags,NaN,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01 Ags,NaN,Anuales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01 Ags,Frijol,Frijol,3358.74626,3201.45101,2290.601174,2409.01026,2251.71501,1120.357124,949.736,949.736,1170.24405
3,01 Ags,Maíz blanco,Maíz blanco,73393.422331,67484.245731,201502.371934,61353.990581,55444.813981,82382.615314,12039.43175,12039.43175,119119.75662
4,01 Ags,NaN,Perennes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Extract key:value for state code

# Filter rows when Cultivo es NaN
df.dropna(subset=['Entidad federativa', 'Entidad federativa y cultivo'], inplace=True)
estado_codigo_nombre = df.loc[df['Cultivo'].isna(), ['Entidad federativa', 'Entidad federativa y cultivo']]
estado_codigo_nombre = estado_codigo_nombre[~estado_codigo_nombre['Entidad federativa y cultivo'].isin(["Perennes", "Anuales"])]
estado_codigo_nombre.columns = ["state_code","state_name"] 

# Split 'codigo' into 'adm_id' and 'state_acrs'
estado_codigo_nombre[['adm_id', 'state_acrs']] = estado_codigo_nombre['state_code'].str.split(' ', expand=True)

estado_codigo_nombre['adm_id'] = estado_codigo_nombre['adm_id'].astype(str)

# add campeche state which is missing

missing_state_data = {
    'state_code': '04 Camp',
    'state_name': 'Campeche',
    'adm_id': '04',
    'state_acrs': 'Camp'
}


missing_state_df = pd.DataFrame([missing_state_data])
estado_codigo_nombre = pd.concat([estado_codigo_nombre,missing_state_df],ignore_index=True)
estado_codigo_nombre['adm_id'] = estado_codigo_nombre['adm_id'].apply(lambda x: 'MX' + x)
# show dataframe
print(estado_codigo_nombre)

# Save key df
estado_codigo_nombre.to_csv('LUT_state.csv', index=False)

   state_code                       state_name adm_id state_acrs
0      01 Ags                   Aguascalientes   MX01        Ags
1       02 BC                  Baja California   MX02         BC
2      03 BCS              Baja California Sur   MX03        BCS
3      05 Coa             Coahuila de Zaragoza   MX05        Coa
4      06 Col                           Colima   MX06        Col
5      07 Chs                          Chiapas   MX07        Chs
6      08 Chi                        Chihuahua   MX08        Chi
7      09 CMX                 Ciudad de México   MX09        CMX
8      10 Dgo                          Durango   MX10        Dgo
9      11 Gto                       Guanajuato   MX11        Gto
10     12 Gro                         Guerrero   MX12        Gro
11     13 Hgo                          Hidalgo   MX13        Hgo
12     14 Jal                          Jalisco   MX14        Jal
13     15 Mex                           México   MX15        Mex
14     16 Mic            